<a href="https://colab.research.google.com/github/LuisPeMoraRod/AI-Laboratories/blob/main/Lab5_MatrixProgrammed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:

# import libraries
import torch


### Ejercicio 1
Implemente la función *calcularTrazaMatriz*, la cual calcula la traza de una matriz usando únicamente operaciones básicas en Pytorch (multiplicación, multiplicación por elemento, matriz identidad, etc), prescindiendo de estructuras de repetición como el for el while. (10 pts) 


Sea una matriz cuadrada *A* de orden *n*, se define la traza de la matriz A,denotada por **tr(A)** como el valor obtenido al sumar todos los elementos de la diagonal principal.
Es decir:

\begin{equation}
  tr(A) = \sum_{i=1}^{n} a_{ii}
\end{equation}

Considerando el siguiente ejemplo:
\begin{pmatrix}
     1 & 5 & 2 \\ 
     0 & 1 & 2 \\
     3 & 1 & 0
    \end{pmatrix}

Se tiene que:

\begin{equation}
  tr(A) = 1 + 1 + 0 = 2
\end{equation}

In [137]:
def get_trace(M: torch.Tensor) -> int:
  diag = torch.diag(M, 0)
  return int(torch.sum(diag))

mat = [[1, 5, 2], [0, 1, 2], [3, 1, 0]]
mat = torch.tensor(mat, dtype=torch.float64)
print(get_trace(mat))


2


### Ejercicio 2
Desarrolle la tranpuesta de una matriz. (10 pts)

In [138]:
def transpuesta(mat):

    # Utilizamos la función view() para intercambiar las dimensiones del tensor
    transpuesta_mat = mat.permute(1, 0)
    
    return transpuesta_mat


# Creamos una matriz de ejemplo
matriz = torch.tensor([[1, 2, 3], [4, 5, 6],[7, 8, 9]])


transpuesta_mat = transpuesta(matriz)


print("------------ Resultados ----------------")
print("Matriz Original")
print(matriz)
print("Matriz Transpuesta")
print(transpuesta_mat)

------------ Resultados ----------------
Matriz Original
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
Matriz Transpuesta
tensor([[1, 4, 7],
        [2, 5, 8],
        [3, 6, 9]])


### Ejercicio 3
Desarrolle la matriz adjunta de una matriz (10 pts)

In [139]:
class Adj:
  # auxiliar variables used to loop through the Tensor
  i = 0
  j = 0
  def __init__(self, M: torch.Tensor):
    self.M = M
    self.rows = len(M)
    self.col = len(M[0])
    self.ones = torch.ones(self.rows, self.col)
    self.adj = self.get_adj_matrix()

  def __str__(self):
    return str(self.adj)

  def get_minor(self, i : int, j : int):
      """
      Return the minor matrix in relation to indexes i and j
      """
      M = torch.cat((self.M[:i],self.M[i+1:])) # delete row i
      M = torch.transpose(M, 0, 1)
      M = torch.cat((M[:j], M[j+1:])) # delete column j
      M = torch.transpose(M, 0, 1)
      return M

  def get_cofactor(self, element) -> int:
    """
    Returns the cofactor for a specific element in the matrix
    """
    minor = self.get_minor(self.i, self.j)
    determinant = torch.det(minor)
    result = (-element)**(self.i+self.j)*determinant #computes cofactor

    #update indexes
    if (self.j + 1 == len(self.M[0])):
      self.i+=1
      self.j = 0
    else:
      self.j+=1

    return result

  def get_cofactors_matrix(self) -> torch.Tensor:
    return self.ones.apply_(self.get_cofactor) #applies get_factor method to every element in ones matrix

  def get_adj_matrix(self) -> torch.Tensor:
    cofactor_matrix = self.get_cofactors_matrix()
    return torch.transpose(cofactor_matrix, 0, 1)

M = [[7, 5, 3, 4], [2, 3, 6, 4], [6, 10, 9, 4], [1, 2, 3, 4]]
M = torch.tensor(M, dtype=torch.float64)
print(f'Original matrix:\n{M}')
adj_matrix = Adj(M)
print(f'\n\nAdjugate matrix:\n{adj_matrix}')

Original matrix:
tensor([[ 7.,  5.,  3.,  4.],
        [ 2.,  3.,  6.,  4.],
        [ 6., 10.,  9.,  4.],
        [ 1.,  2.,  3.,  4.]], dtype=torch.float64)


Adjugate matrix:
tensor([[ -72.,  -72.,   36.,  108.],
        [  36.,  144.,  -72., -108.],
        [  12., -132.,   12.,  108.],
        [  -9.,   45.,   18., -135.]])


### Ejercicio 4
Desarrolle la inversa de una matriz. Debe validar si es invertible o no. (20 pts)

In [143]:

# Metodo para determinar la inversa de una matriz

def matriz_inversa(matrix):
  
  # Cálculo del determinante
  determinante = torch.det(matrix)

  # Verificación de si tiene inversa
  if  determinante == 0:
    return "La matriz no tiene inversa"

  # Cálculo de la inversa usando la fórmula general
  m_transpuesta = transpuesta(matrix)
  m_adj = Adj(m_transpuesta)
  m_inversa = m_adj.adj/determinante

  return m_inversa

matriz = torch.tensor([[2, -2, 2], [2, 1, 0],[3, -2, 2]])
matriz_tensor = torch.tensor(matriz, dtype=torch.float64)
mat_inversa = matriz_inversa(matriz_tensor)
print("------------ Método programado ----------------")
print("Matriz Original")
print(matriz)
print("Matriz Inversa")
print(mat_inversa)

------------ Método programado ----------------
Matriz Original
tensor([[ 2, -2,  2],
        [ 2,  1,  0],
        [ 3, -2,  2]])
Matriz Inversa
tensor([[-1.0000,  2.0000,  3.5000],
        [-0.0000,  1.0000,  1.0000],
        [ 1.0000, -2.0000, -3.0000]])


<ipython-input-143-2bdc5f84b2ed>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matriz_tensor = torch.tensor(matriz, dtype=torch.float64)


### Ejercicio 5
La Matriz de covarianza. (50 pts) 

In [141]:
class CovMat:
  # auxiliar variables used to loop through the Tensor

  def __init__(self, M: torch.Tensor):
    self.rows = len(M)
    self.cols = len(M[0])
    self.ones = torch.ones(self.rows, self.cols)
    self.M = M
    self.means_mat = self.get_means_matrix() #matrix that contains the mean of every variable
    self.cov_mat = self.get_covariances_matrix()

  def __str__(self):
    return str(self.cov_mat)

  def get_means_matrix(self):
    means_mat = torch.mean(M, dim=1)
    return means_mat.reshape(-1,1) #returns transpose

  def get_covariances_matrix(self):
    matrix = self.M

    ones = torch.ones(self.rows, self.cols)
    means = self.means_mat * ones # matrix of dimension nxm where every row corresponds to the mean of that variable

    #apply formula
    mat_minus_u = matrix - means
    t_mat_minus_u = torch.transpose(mat_minus_u, 0, 1) #tranponse to multiply with mirrored elements
    cov_mat = torch.matmul(mat_minus_u, t_mat_minus_u)
    cov_mat = cov_mat * 1/(self.cols-1)

    return cov_mat
  


M = [[2, 4, 6, 8], [4, 8, 12, 16], [12, 10, 5, 9]]
M = torch.tensor(M, dtype=torch.float64)

cov_mat = CovMat(M)
print(cov_mat)
  

tensor([[ 6.6667, 13.3333, -4.6667],
        [13.3333, 26.6667, -9.3333],
        [-4.6667, -9.3333,  8.6667]], dtype=torch.float64)
